In [ ]:

######## snakemake preamble start (automatically inserted, do not edit) ########
import sys; sys.path.extend(['/fh/fast/bloom_j/software/miniconda3/envs/barcoded_flu_pdmH1N1/lib/python3.8/site-packages', '/fh/fast/bloom_j/computational_notebooks/dbacsik/2019/flu_pdmH1N1_barcode_hashing/notebooks']); import pickle; snakemake = pickle.loads(b'\x80\x04\x95sC\x00\x00\x00\x00\x00\x00\x8c\x10snakemake.script\x94\x8c\tSnakemake\x94\x93\x94)\x81\x94}\x94(\x8c\x05input\x94\x8c\x0csnakemake.io\x94\x8c\nInputFiles\x94\x93\x94)\x81\x94(\x8c_results/viral_tags_bcs_in_cells/scProgenyProduction_trial3_cell_barcodes_with_viral_tags.csv.gz\x94\x8cSresults/viral_fastq10x/scProgenyProduction_trial3_viral_bc_by_cell_corrected.csv.gz\x94\x8cLresults/viral_fastq10x/scProgenyProduction_trial3_viral_genes_by_cell.csv.gz\x94e}\x94(\x8c\x06_names\x94}\x94(\x8c\x15viral_tag_by_cell_csv\x94K\x00N\x86\x94\x8c\x1eviral_bc_by_cell_corrected_csv\x94K\x01N\x86\x94\x8c\x17viral_genes_by_cell_csv\x94K\x02N\x86\x94u\x8c\x12_allowed_overrides\x94]\x94(\x8c\x05index\x94\x8c\x04sort\x94eh\x18\x8c\tfunctools\x94\x8c\x07partial\x94\x93\x94h\x06\x8c\x19Namedlist._used_attribute\x94\x93\x94\x85\x94R\x94(h\x1e)}\x94\x8c\x05_name\x94h\x18sNt\x94bh\x19h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x19sNt\x94bh\x10h\nh\x12h\x0bh\x14h\x0cub\x8c\x06output\x94h\x06\x8c\x0bOutputFiles\x94\x93\x94)\x81\x94(\x8cOresults/viral_fastq10x/scProgenyProduction_trial3_valid_viral_bc_by_cell.csv.gz\x94\x8cRresults/viral_fastq10x/scProgenyProduction_trial3_valid_viral_barcodes_by_cell.pdf\x94e}\x94(h\x0e}\x94(\x8c\x18valid_viral_barcodes_csv\x94K\x00N\x86\x94\x8c\x04plot\x94K\x01N\x86\x94uh\x16]\x94(h\x18h\x19eh\x18h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x18sNt\x94bh\x19h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x19sNt\x94bh0h,h2h-ub\x8c\x06params\x94h\x06\x8c\x06Params\x94\x93\x94)\x81\x94]\x94(\x8c\x05fluHA\x94\x8c\x05fluNA\x94ea}\x94(h\x0e}\x94\x8c\x14barcoded_viral_genes\x94K\x00N\x86\x94sh\x16]\x94(h\x18h\x19eh\x18h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x18sNt\x94bh\x19h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x19sNt\x94bhFhAub\x8c\twildcards\x94h\x06\x8c\tWildcards\x94\x93\x94)\x81\x94\x8c\x1ascProgenyProduction_trial3\x94a}\x94(h\x0e}\x94\x8c\x04expt\x94K\x00N\x86\x94sh\x16]\x94(h\x18h\x19eh\x18h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x18sNt\x94bh\x19h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x19sNt\x94b\x8c\x04expt\x94hUub\x8c\x07threads\x94K\x01\x8c\tresources\x94h\x06\x8c\tResources\x94\x93\x94)\x81\x94(K\x01K\x01e}\x94(h\x0e}\x94(\x8c\x06_cores\x94K\x00N\x86\x94\x8c\x06_nodes\x94K\x01N\x86\x94uh\x16]\x94(h\x18h\x19eh\x18h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x18sNt\x94bh\x19h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x19sNt\x94bhkK\x01hmK\x01ub\x8c\x03log\x94h\x06\x8c\x03Log\x94\x93\x94)\x81\x94\x8cJresults/logs/valid_viral_barcodes_by_cell_scProgenyProduction_trial3.ipynb\x94a}\x94(h\x0e}\x94\x8c\x08notebook\x94K\x00N\x86\x94sh\x16]\x94(h\x18h\x19eh\x18h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x18sNt\x94bh\x19h\x1ch\x1e\x85\x94R\x94(h\x1e)}\x94h"h\x19sNt\x94bh\x7fh|ub\x8c\x06config\x94}\x94(\x8c\x08max_cpus\x94K\x10\x8c\x07log_dir\x94\x8c\x0cresults/logs\x94\x8c\x0cfastq10x_dir\x94\x8c\x10results/fastq10x\x94\x8c\npacbio_dir\x94\x8c\x0eresults/pacbio\x94\x8c\x0emkfastq10x_dir\x94\x8c\x1fresults/fastq10x/mkfastq_output\x94\x8c\ngenome_dir\x94\x8c\x0fresults/genomes\x94\x8c\trefgenome\x94\x8c\x19results/genomes/refgenome\x94\x8c\x14aligned_fastq10x_dir\x94\x8c\x18results/aligned_fastq10x\x94\x8c\x12viral_fastq10x_dir\x94\x8c\x16results/viral_fastq10x\x94\x8c\x1bviral_tags_bcs_in_cells_dir\x94\x8c\x1fresults/viral_tags_bcs_in_cells\x94\x8c\x11viral_progeny_dir\x94\x8c\x15results/viral_progeny\x94\x8c\x13align_and_parse_dir\x94\x8c#results/pacbio/CA09_align_and_parse\x94\x8c\x0fcell_genome_ftp\x94\x8cmftp://ftp.ensembl.org/pub/release-98/fasta/canis_familiaris/dna/Canis_familiaris.CanFam3.1.dna.toplevel.fa.gz\x94\x8c\x0ccell_gtf_ftp\x94\x8c^ftp://ftp.ensembl.org/pub/release-98/gtf/canis_familiaris/Canis_familiaris.CanFam3.1.98.gtf.gz\x94\x8c\x0cviral_genome\x94\x8c!data/flu_sequences/flu-CA09.fasta\x94\x8c\tviral_gtf\x94\x8c\x1fdata/flu_sequences/flu-CA09.gtf\x94\x8c\rviral_genbank\x94\x8c\x1edata/flu_sequences/flu-CA09.gb\x94\x8c\x14viral_tag_identities\x94\x8c+data/flu_sequences/flu-CA09_viral_tags.yaml\x94\x8c\x0fviral_amplicons\x94\x8c.data/flu_sequences/pacbio_amplicons/fluCA09.gb\x94\x8c\x11amplicon_features\x94\x8c9data/flu_sequences/pacbio_amplicons/fluCA09_features.yaml\x94\x8c\x1eamplicon_linearization_primers\x94\x8c=data/flu_sequences/pacbio_amplicons/linearization_primers.tsv\x94\x8c\x15amplicon_to_reference\x94\x8c=data/flu_sequences/pacbio_amplicons/amplicon_to_reference.csv\x94\x8c\x0escoreGapNoncan\x94J\xfc\xff\xff\xff\x8c\x0cscoreGapGCAG\x94J\xfc\xff\xff\xff\x8c\x0cscoreGapATAC\x94J\xfc\xff\xff\xff\x8c\x14cb_whitelist_10x_url\x94\x8ckhttps://github.com/10XGenomics/cellranger/raw/master/lib/python/cellranger/barcodes/3M-february-2018.txt.gz\x94\x8c\x10cb_whitelist_10x\x94\x8c-results/aligned_fastq10x/cb_whitelist_10x.txt\x94\x8c\ncb_len_10x\x94K\x10\x8c\x0bumi_len_10x\x94K\x0c\x8c\x13infection_threshold\x94G?\x84z\xe1G\xae\x14{\x8c\rviral_tag_fdr\x94G?\x1a6\xe2\xeb\x1cC-\x8c\x1dviral_barcode_upstream_length\x94K\x1c\x8c\x16viral_barcode_mismatch\x94K\x00\x8c\x12viral_barcode_minq\x94K\x1e\x8c\x0cviral_bc_fdr\x94G?\x1a6\xe2\xeb\x1cC-\x8c\x0eccs_min_length\x94K2\x8c\x0eccs_max_length\x94M\x88\x13\x8c\nccs_min_rq\x94G?\xef\xf7\xce\xd9\x16\x87+\x8c\x0bexperiments\x94}\x94(\x8c\x0ehashing_lowMOI\x94}\x94(\x8c\x0bdescription\x94X*\x01\x00\x00Single-cell transcriptomics using the wildtype and dblySyn viral tag variants. Infection volume was chosen based on the results of `hashing_trial1` and flow cytometry. The incolum volume for the wildtype tag variant was about 12-fold higher, and for the dblSyn tag variant was about 24-fold higher.\x94\x8c\tlab_notes\x94\x8c0https://benchling.com/s/etr-W8urOmOAQ7L6U4HAXMNy\x94\x8c\rexpect_ncells\x94M\x88\x13\x8c\x0ftranscriptomics\x94}\x94(\x8c\x08datetime\x94\x8c\x04date\x94\x93\x94C\x04\x07\xe4\x06\x02\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-GA-B3\x94\x8c\nbcl_folder\x94\x8cG/home/solexa/ngs/illumina/bloom_lab/200602_M03100_0578_000000000-J3464/\x94\x8c\x04lane\x94\x8c\x01*\x94\x8c\x10index_sequencing\x94\x8c\x06single\x94uh\xd6C\x04\x07\xe4\x07\x01\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-GA-B3\x94\x8c\nbcl_folder\x94\x8c^/shared/ngs/illumina/agreaney/200701_D00300_0988_BHGFGMBCX3/raw/200701_D00300_0988_BHGFGMBCX3/\x94\x8c\x04lane\x94K\x02\x8c\x10index_sequencing\x94\x8c\x06single\x94uh\xd6C\x04\x07\xe4\x08\x1a\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-GA-B3\x94\x8c\nbcl_folder\x94\x8ci/shared/ngs/illumina/bloom_lab/bloom_lab/200826_D00300_1040_BHHGLWBCX3/raw/200826_D00300_1040_BHHGLWBCX3/\x94\x8c\x04lane\x94h\xe0\x8c\x10index_sequencing\x94\x8c\x06single\x94uh\xd6C\x04\x07\xe5\x03\x0c\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-GA-B3\x94\x8c\nbcl_folder\x94\x8c_/shared/ngs/illumina/bloom_lab/210312_D00300_1192_BHKHNYBCX3/raw/210312_D00300_1192_BHKHNYBCX3/\x94\x8c\x04lane\x94K\x01\x8c\x10index_sequencing\x94\x8c\x06single\x94uuu\x8c\x0fhashing_highMOI\x94}\x94(\x8c\x0bdescription\x94\x8c\x9aSingle-cell transcriptomics using the wildtype and dblSyn viral tag variants at a high MOI. Infection volume was based on the results of `hashing_trial2`.\x94\x8c\tlab_notes\x94\x8c0https://benchling.com/s/etr-ToK66qT0Mfmf6FsexKo4\x94\x8c\rexpect_ncells\x94M\x88\x13\x8c\x0ftranscriptomics\x94}\x94(h\xd6C\x04\x07\xe4\x07\x18\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-GA-B5\x94\x8c\nbcl_folder\x94\x8cB/shared/ngs/illumina/bloom_lab/200724_M03100_0593_000000000-J33YK/\x94\x8c\x04lane\x94h\xe0\x8c\x10index_sequencing\x94\x8c\x06single\x94uh\xd6C\x04\x07\xe4\x08\x1a\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-GA-B5\x94\x8c\nbcl_folder\x94\x8ci/shared/ngs/illumina/bloom_lab/bloom_lab/200826_D00300_1040_BHHGLWBCX3/raw/200826_D00300_1040_BHHGLWBCX3/\x94\x8c\x04lane\x94h\xe0\x8c\x10index_sequencing\x94\x8c\x06single\x94uh\xd6C\x04\x07\xe5\x03\x0c\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-GA-B5\x94\x8c\nbcl_folder\x94\x8c_/shared/ngs/illumina/bloom_lab/210312_D00300_1192_BHKHNYBCX3/raw/210312_D00300_1192_BHKHNYBCX3/\x94\x8c\x04lane\x94K\x02\x8c\x10index_sequencing\x94\x8c\x06single\x94uu\x8c\x17pacbio_viral_sequencing\x94}\x94\x8c\x1c2020-10-20_all_segments_run1\x94\x8cw/fh/fast/bloom_j/SR/ngs/pacbio/201019_BernadetaDadonaite/r54228_20201020_192758/1_A01/m54228_201020_194205.subreads.bam\x94su\x8c\x1ascProgenyProduction_trial1\x94}\x94(\x8c\x0bdescription\x94\x8c\xe7Single-cell transcriptomics performed on low MOI-infected sample. Innoculum volumes based on `hashing_trial2` data. Loaded all collected cells from two 24-well wells, each infected with either wildtype or dblySn viral tag variants.\x94\x8c\tlab_notes\x94\x8c0https://benchling.com/s/etr-20lOO7l1FdEJJIpxV6vD\x94\x8c\rexpect_ncells\x94M\x10\'\x8c\x0ftranscriptomics\x94}\x94h\xd6C\x04\x07\xe4\t\x19\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-GA-B8\x94\x8c\nbcl_folder\x94\x8c_/shared/ngs/illumina/bloom_lab/200925_D00300_1065_AHHL7NBCX3/raw/200925_D00300_1065_AHHL7NBCX3/\x94\x8c\x04lane\x94K\x02\x8c\x10index_sequencing\x94\x8c\x06single\x94us\x8c\x0eviral_barcodes\x94}\x94(\x8c\x0bsupernatant\x94}\x94(\x8c\x02wt\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8a/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-Sup-fluHA-A_S1_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8a/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-Sup-fluHA-B_S2_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8b/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-Sup-fluNA-A_S17_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8b/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-Sup-fluNA-B_S18_L001_R1_001.fastq.gz\x94suu\x8c\x03syn\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8e/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-Sup-fluHA-A_S3_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8e/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-Sup-fluHA-B_S4_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8f/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-Sup-fluNA-A_S19_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8f/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-Sup-fluNA-B_S20_L001_R1_001.fastq.gz\x94suuu\x8c\x10second_infection\x94}\x94(\x8c\x02wt\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x90/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-SecondInf-fluHA-A_S5_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x90/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-SecondInf-fluHA-B_S6_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x91/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-SecondInf-fluNA-A_S21_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x91/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-WT-SecondInf-fluNA-B_S22_L001_R1_001.fastq.gz\x94suu\x8c\x03syn\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x94/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-SecondInf-fluHA-A_S7_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x94/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-SecondInf-fluHA-B_S8_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x95/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-SecondInf-fluNA-A_S23_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x95/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial1-dblSyn-SecondInf-fluNA-B_S24_L001_R1_001.fastq.gz\x94suuuuu\x8c\x1ascProgenyProduction_trial2\x94}\x94(\x8c\x0bdescription\x94\x8c\xe7Single-cell transcriptomics performed on low MOI-infected sample. Innoculum volumes based on `hashing_trial2` data. Loaded all collected cells from two 24-well wells, each infected with either wildtype or dblySn viral tag variants.\x94\x8c\tlab_notes\x94\x8c0https://benchling.com/s/etr-73UD4Mx8uXn1xcxlV1ym\x94\x8c\rexpect_ncells\x94M\x10\'\x8c\x0ftranscriptomics\x94}\x94h\xd6C\x04\x07\xe4\t\x19\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-GA-B9\x94\x8c\nbcl_folder\x94\x8c_/shared/ngs/illumina/bloom_lab/200925_D00300_1065_AHHL7NBCX3/raw/200925_D00300_1065_AHHL7NBCX3/\x94\x8c\x04lane\x94h\xe0\x8c\x10index_sequencing\x94\x8c\x06single\x94us\x8c\x0eviral_barcodes\x94}\x94(\x8c\x0bsupernatant\x94}\x94(\x8c\x02wt\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8a/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-WT-Sup-fluHA-A_S9_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8b/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-WT-Sup-fluHA-B_S10_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8b/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-WT-Sup-fluNA-A_S25_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8b/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-WT-Sup-fluNA-B_S26_L001_R1_001.fastq.gz\x94suu\x8c\x03syn\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8f/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-dblSyn-Sup-fluHA-A_S11_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8f/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-dblSyn-Sup-fluHA-B_S12_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8f/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-dblSyn-Sup-fluNA-A_S27_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x8f/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-dblSyn-Sup-fluNA-B_S28_L001_R1_001.fastq.gz\x94suuu\x8c\x10second_infection\x94}\x94(\x8c\x02wt\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x91/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-WT-SecondInf-fluHA-A_S13_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x91/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-WT-SecondInf-fluHA-B_S14_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x91/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-WT-SecondInf-fluNA-A_S29_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x91/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-WT-SecondInf-fluNA-B_S30_L001_R1_001.fastq.gz\x94suu\x8c\x03syn\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x95/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-dblSyn-SecondInf-fluHA-A_S15_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x95/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-dblSyn-SecondInf-fluHA-B_S16_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x95/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-dblSyn-SecondInf-fluNA-A_S31_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe4\n\r\x94\x85\x94R\x94\x8c\x95/shared/ngs/illumina/bloom_lab/201013_M03100_0625_000000000-JB2KP/Data/Intensities/BaseCalls/Trial2-dblSyn-SecondInf-fluNA-B_S32_L001_R1_001.fastq.gz\x94suuuuu\x8c\x1ascProgenyProduction_trial3\x94}\x94(\x8c\x0bdescription\x94X\x1b\x01\x00\x00Single-cell transcriptomics performed on even lower MOI-infected sample. Innoculum volumes based on `scProgenyProduction_trial1` and `scProgenyProduction_trial2` data. Loaded all collected cells from two 24-well wells, each infected with either wildtype or dblySn viral tag variants.\x94\x8c\tlab_notes\x94\x8c0https://benchling.com/s/etr-N4laiIAy3AUePJdig7wR\x94\x8c\rexpect_ncells\x94M\x10\'\x8c\x0ftranscriptomics\x94}\x94(h\xd6C\x04\x07\xe5\x03\x0c\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-TT-A3\x94\x8c\nbcl_folder\x94\x8c_/shared/ngs/illumina/bloom_lab/210312_D00300_1192_BHKHNYBCX3/raw/210312_D00300_1192_BHKHNYBCX3/\x94\x8c\x04lane\x94K\x01\x8c\x10index_sequencing\x94\x8c\x06single\x94uh\xd6C\x04\x07\xe5\x04\t\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-TT-A3\x94\x8c\nbcl_folder\x94\x8c_/shared/ngs/illumina/bloom_lab/210409_D00300_1210_BHKC7KBCX3/raw/210409_D00300_1210_BHKC7KBCX3/\x94\x8c\x04lane\x94h\xe0\x8c\x10index_sequencing\x94\x8c\x06single\x94uh\xd6C\x04\x07\xe5\x08\x19\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-TT-A3\x94\x8c\nbcl_folder\x94\x8c_/shared/ngs/illumina/bloom_lab/210825_D00300_1310_BHMH5TBCX3/raw/210825_D00300_1310_BHMH5TBCX3/\x94\x8c\x04lane\x94h\xe0\x8c\x10index_sequencing\x94\x8c\x04none\x94uh\xd6C\x04\x07\xe5\t\x0f\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-TT-A3\x94\x8c\nbcl_folder\x94\x8cY/shared/ngs/illumina/bloom_lab/210915_VH00699_2_AAAMCL7M5/raw/210915_VH00699_2_AAAMCL7M5/\x94\x8c\x04lane\x94h\xe0\x8c\x10index_sequencing\x94\x8c\x04none\x94uu\x8c\x17pacbio_viral_sequencing\x94}\x94\x8c\x1c2021-08-05_all_segments_run1\x94\x8cp/fh/fast/bloom_j/SR/ngs/pacbio/210730_SequelIIe/r64272e_20210730_182729/1_A01/m64272e_210730_193026.subreads.bam\x94s\x8c\x0eviral_barcodes\x94}\x94(\x8c\x0bsupernatant\x94}\x94(\x8c\x02wt\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x90/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/WT-sup-fluHA-rep1_S9_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x91/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/WT-sup-fluHA-rep2_S13_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x90/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/WT-sup-fluNA-rep1_S1_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x86/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/WT-sup-fluNA-rep2_S1_L001_R1_001.fastq.gz\x94suu\x8c\x03syn\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x95/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/dblSyn-sup-fluHA-rep1_S10_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x95/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/dblSyn-sup-fluHA-rep2_S14_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x94/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/dblSyn-sup-fluNA-rep1_S2_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x94/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/dblSyn-sup-fluNA-rep2_S6_L001_R1_001.fastq.gz\x94suuu\x8c\x10second_infection\x94}\x94(\x8c\x02wt\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x85/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/WT-SI-fluHA-rep1_S2_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x85/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/WT-SI-fluHA-rep2_S3_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x86/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/WT-SI-fluNA-rep1_S11_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x86/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/WT-SI-fluNA-rep2_S12_L001_R1_001.fastq.gz\x94suu\x8c\x03syn\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x89/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/dblSyn-SI-fluHA-rep1_S4_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x89/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/dblSyn-SI-fluHA-rep2_S5_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xd6C\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x8a/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/dblSyn-SI-fluNA-rep1_S13_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xd6C\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x8a/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/dblSyn-SI-fluNA-rep2_S14_L001_R1_001.fastq.gz\x94suuuuuuu\x8c\x04rule\x94\x8c\x1cvalid_viral_barcodes_by_cell\x94\x8c\x0fbench_iteration\x94N\x8c\tscriptdir\x94\x8c[/fh/fast/bloom_j/computational_notebooks/dbacsik/2019/flu_pdmH1N1_barcode_hashing/notebooks\x94ub.'); from snakemake.logging import logger; logger.printshellcmds = False; import os; os.chdir('/fh/fast/bloom_j/computational_notebooks/dbacsik/2019/flu_pdmH1N1_barcode_hashing');
######## snakemake preamble end #########


# Viral barcode in each cell
This notebook filters viral barcodes in each cell in the 10X transcriptome data

## Notebook setup
Import python modules:

In [ ]:
from IPython.display import display

from dms_variants.constants import CBPALETTE

import pandas as pd

import plotnine as p9

import scipy

import statsmodels.stats.multitest

Get `snakemake` variables [as described here](https://snakemake.readthedocs.io/en/stable/snakefiles/rules.html#jupyter-notebook-integration):

In [ ]:
viral_tag_by_cell_csv = (snakemake
                         .input
                         .viral_tag_by_cell_csv)
viral_bc_by_cell_corrected_csv = (snakemake
                                  .input
                                  .viral_bc_by_cell_corrected_csv)
viral_genes_by_cell_csv = snakemake.input.viral_genes_by_cell_csv
valid_viral_barcodes_csv = (
    snakemake
    .output
    .valid_viral_barcodes_csv)
plot = snakemake.output.plot
expt = snakemake.wildcards.expt
barcoded_viral_genes = snakemake.params.barcoded_viral_genes

Set base plot style:

In [ ]:
p9.theme_set(p9.theme_classic())

## Organize data

Read the viral barcode UMI counts data into a pandas dataframe:

In [ ]:
viral_bc_counts = pd.read_csv(viral_bc_by_cell_corrected_csv)
viral_bc_counts = (viral_bc_counts
                   .rename(columns={'count': 'viral_bc_UMIs'}))
assert set(viral_bc_counts['gene']) == set(barcoded_viral_genes), \
       "Barcoded genes in barcode counts do not match expectation."
display(viral_bc_counts)

Read the total number of UMIs per cell into a pandas dataframe. Only keep relevant columns.

In [ ]:
all_cells = pd.read_csv(viral_tag_by_cell_csv)
all_cells = all_cells[['cell_barcode',
                       'infected',
                       'infecting_viral_tag',
                       'total_UMIs',
                       'viral_UMIs',
                       'cellular_UMIs',
                       'frac_viral_UMIs']]
display(all_cells)

Sanity check that `total_UMIs` is equal to `viral_UMIs + cellular_UMIs`:

In [ ]:
assert bool((all_cells['total_UMIs'] ==
             all_cells['viral_UMIs'] +
             all_cells['cellular_UMIs'])
            .all()), "UMI counts do not add up"

Read the gene presence/absence data

In [ ]:
viral_genes_by_cell = pd.read_csv(viral_genes_by_cell_csv)
display(viral_genes_by_cell)

Merge dataframes:

In [ ]:
viral_bc_frac = pd.merge(
    left=pd.concat([all_cells.assign(gene=gene)
                    for gene in barcoded_viral_genes]),
    right=viral_bc_counts,
    how='outer',
    on=['cell_barcode', 'gene'],
    validate='one_to_many')
assert (viral_bc_frac['cell_barcode'].unique() ==
        all_cells['cell_barcode'].unique()).all(), \
       "Cell barcodes in merged dataframe don't " \
       "match barcodes in source data."
assert (viral_bc_frac['viral_barcode'].nunique() ==
        viral_bc_counts['viral_barcode'].nunique()), \
       "Number of viral barcodes in merged dataframe don't " \
       "match number of barcodes in source data."

# Make `infecting_viral_tag` column ordered category
viral_bc_frac['infecting_viral_tag'] = (viral_bc_frac['infecting_viral_tag']
                                        .astype('category')
                                        .cat
                                        .reorder_categories(['none',
                                                             'wt',
                                                             'syn',
                                                             'both']))

viral_bc_frac = pd.merge(
    left=viral_bc_frac,
    right=viral_genes_by_cell,
    on=['cell_barcode', 'gene'],
    how='left',
    validate='many_to_one'
)

display(viral_bc_frac)

Calculate **each barcode's** fraction of all UMIs per cell:

In [ ]:
viral_bc_frac = (
    viral_bc_frac
    .assign(viral_bc_UMIs=lambda x: (x['viral_bc_UMIs']
                                     .fillna(0)
                                     .astype(int, errors='raise'))))


viral_bc_frac['frac_viral_bc_UMIs'] = (
    viral_bc_frac['viral_bc_UMIs'] /
    viral_bc_frac['total_UMIs'])

display(viral_bc_frac)

## Filtering
Now, I will filter out viral barcodes in each infected cell. A valid viral barcode meets the following criteria:
1. The cell is infected
2. The cell expresses the barcoded gene
3. There are at least 2 UMIs with the viral barcode in that cell-gene
4. The viral barcode is present at a frequency greater than some cutoff percentile observed in uninfected cells
5. The viral barocde contributes at least 10% of the viral barcode UMIs within the cell-gene

In [ ]:
filter_query_list = list()

### Infected cells
Valid viral barcodes must come from infected cells:

In [ ]:
infected_rank = (viral_bc_frac
                 [['cell_barcode',
                   'gene',
                   'infected',
                   'frac_viral_bc_UMIs',]]
                 .copy()
                 .drop_duplicates())
infected_rank['rank'] = (infected_rank
                         .groupby('gene')
                         ['frac_viral_bc_UMIs']
                         .rank(ascending=False, method='first'))

fig = (p9.ggplot(infected_rank,
                 p9.aes(x='rank',
                        y='frac_viral_bc_UMIs',
                        color='infected')) +
       p9.geom_point(alpha=.3) +
       p9.facet_grid('gene~infected') +
       p9.scale_y_log10() +
       p9.ggtitle('viral bc UMIs in infected and uninfected cells\n'
                  f'{expt}') +
       p9.labs(x='viral bc frequency rank') +
       p9.theme(figure_size=(6, 4),
                plot_title=p9.element_text(size=9),
                axis_title=p9.element_text(size=9),
                legend_title=p9.element_text(size=9),
                legend_title_align='center') +
       p9.scale_color_manual(CBPALETTE[0:]))
display(fig)

Add infection status to filter query:

In [ ]:
filter_query_list.append('(infected == "infected")')
filter_query = " and ".join(filter_query_list)
print("The filter query is:  "
      f"{filter_query}")

### Gene expression
Plot barcode UMIs per viral barcode in infected cells, annotated with gene expression:

In [ ]:
gene_expressed_rank = (viral_bc_frac
                       .query(filter_query)
                       [['cell_barcode',
                         'gene',
                         'gene_present',
                         'frac_viral_bc_UMIs',]]
                       .copy()
                       .drop_duplicates())
gene_expressed_rank['rank'] = (gene_expressed_rank
                               .groupby('gene')
                               ['frac_viral_bc_UMIs']
                               .rank(ascending=False, method='first'))

fig = (p9.ggplot(gene_expressed_rank,
                 p9.aes(x='rank',
                        y='frac_viral_bc_UMIs',
                        color='gene_present')) +
       p9.geom_point(alpha=0.5) +
       p9.facet_grid('gene~') +
       p9.scale_y_log10() +
       p9.ggtitle('viral bc UMIs on expressed/missing genes\n'
                  'in infected cells\n'
                  f'{expt}') +
       p9.labs(x='viral bc frequency rank') +
       p9.theme(figure_size=(3, 4),
                plot_title=p9.element_text(size=9),
                axis_title=p9.element_text(size=9),
                legend_title=p9.element_text(size=9),
                legend_title_align='center') +
       p9.scale_color_manual(CBPALETTE[0:]))
display(fig)

Add gene expression to filter query:

In [ ]:
filter_query_list.append('(gene_present == True)')
filter_query = " and ".join(filter_query_list)
print("The filter query is:  "
      f"{filter_query}")

### Absolute abundance
Label and remove viral barcodes with only 1 UMIs in a given cell:

In [ ]:
UMI_limit = 1
viral_bc_frac['above_UMI_limit'] = viral_bc_frac['viral_bc_UMIs'] > UMI_limit

UMI_limit_rank = (viral_bc_frac
                  .query(filter_query)
                  [['cell_barcode',
                    'gene',
                    'above_UMI_limit',
                    'viral_bc_UMIs',]]
                  .copy()
                  .drop_duplicates())
UMI_limit_rank['rank'] = (UMI_limit_rank
                          .groupby('gene')
                          ['viral_bc_UMIs']
                          .rank(ascending=False, method='first'))

fig = (p9.ggplot(UMI_limit_rank,
                 p9.aes(x='rank',
                        y='viral_bc_UMIs',
                        color='above_UMI_limit')) +
       p9.geom_point(alpha=0.5) +
       p9.geom_hline(yintercept=UMI_limit, linetype='dashed', color=CBPALETTE[2]) +
       p9.facet_grid('gene~') +
       p9.scale_y_log10() +
       p9.ggtitle('viral barcode UMI threshold\n'
                  'in infected cells expressing barcoded gene\n'
                  f'{expt}') +
       p9.labs(x='viral bc UMI rank') +
       p9.theme(figure_size=(3, 4),
                plot_title=p9.element_text(size=9),
                axis_title=p9.element_text(size=9),
                legend_title=p9.element_text(size=9),
                legend_title_align='center') +
       p9.scale_color_manual(CBPALETTE[0:]))
display(fig)

Add absolute abundance to filter query:

In [ ]:
filter_query_list.append('(above_UMI_limit == True)')
filter_query = " and ".join(filter_query_list)
print("The filter query is:  "
      f"{filter_query}")

### Background frequency
Find the 99th percentile of individual viral barcode frequency in uninfected cells. Label and filter out viral barcodes that are less than this frequency.

In [ ]:
limit_percentile = 0.99

uninfected_background_frac = {}
for gene in barcoded_viral_genes:
    limit = (viral_bc_frac
             .query('(infected == "uninfected") '
                    f'and (gene == "{gene}")')
             ['frac_viral_bc_UMIs']
             .quantile(limit_percentile))
    uninfected_background_frac[f'{gene}'] = limit
    print(f'The limit for a viral barcode on {gene} is {limit:.5f}.')
    
# Convert to DataFrame for plotting
uninfected_background_frac = (
    pd.DataFrame.from_dict(uninfected_background_frac,
                           orient='index',
                           columns=['uninf_background'])
    .reset_index()
    .rename(columns={'index': 'gene'}))

fig = (p9.ggplot((viral_bc_frac
                  .query('infected == "uninfected"')),
                 p9.aes(x='frac_viral_bc_UMIs')) +
       p9.geom_histogram(bins=50, fill="#3a3a3a") +
       p9.geom_vline(uninfected_background_frac,
                     p9.aes(xintercept='uninf_background'),
                     linetype='dashed',
                     color='#3A3B3C',
                     size=0.5) +
       p9.facet_grid('infected~gene') +
       p9.ggtitle('viral barcode fraction in uninfected cells\n'
                  f'{expt}') +
       p9.labs(x='viral barcode frequency',) +
       p9.theme(figure_size=(5, 1),
                plot_title=p9.element_text(size=10),
                axis_title=p9.element_text(size=10),
                axis_text=p9.element_text(rotation=45),
                legend_title=p9.element_text(size=9),
                legend_title_align='center'))
display(fig)

In [ ]:
viral_bc_frac = pd.merge(left=viral_bc_frac,
                         right=uninfected_background_frac,
                         on='gene',
                         how='outer',
                         validate='many_to_one')
viral_bc_frac['above_uninf_background'] = (viral_bc_frac['frac_viral_bc_UMIs'] >
                                           viral_bc_frac['uninf_background'])

In [ ]:
background_rank = (viral_bc_frac
                   .query(filter_query)
                   [['cell_barcode',
                     'gene',
                     'above_uninf_background',
                     'frac_viral_bc_UMIs',]]
                   .copy()
                   .drop_duplicates())
background_rank['rank'] = (background_rank
                           .groupby('gene')
                           ['frac_viral_bc_UMIs']
                           .rank(ascending=False, method='first'))

fig = (p9.ggplot(background_rank,
                 p9.aes(x='rank',
                        y='frac_viral_bc_UMIs',
                        color='above_uninf_background')) +
       p9.geom_point(alpha=0.5) +
       p9.geom_hline(uninfected_background_frac,
                     p9.aes(yintercept='uninf_background'),
                     linetype='dashed',
                     color=CBPALETTE[2]) +
       p9.facet_grid('gene~') +
       p9.scale_y_log10() +
       p9.ggtitle(f'{expt}') +
       p9.labs(x='viral bc frequency rank') +
       p9.theme(figure_size=(3, 4),
                plot_title=p9.element_text(size=9),
                axis_title=p9.element_text(size=9),
                legend_title=p9.element_text(size=9),
                legend_title_align='center') +
       p9.scale_color_manual(CBPALETTE[0:]))
display(fig)

Add background frequency to filter query:

In [ ]:
filter_query_list.append('(above_uninf_background == True)')
filter_query = " and ".join(filter_query_list)
print("The filter query is:  "
      f"{filter_query}")

### Within-cell proportion
Require a viral barcode to contribute some threshold proportion of the UMIs within each cell-gene.

Calculate the proportion of UMIs that come from each viral barcode within an infected cell. For example, in a cell with only one viral barcode, that viral barcode would have a proportion of 1; a cell with a 70%/30% split of two viral barcodes would have proportions of 0.7 and 0.3.

In [ ]:
within_cell_proportion_df = (
    viral_bc_frac
    .query('infected == "infected" and '
           'viral_bc_UMIs > 0')
    [['cell_barcode',
      'gene',
      'viral_barcode',
      'viral_bc_UMIs']]
    .copy()
    .drop_duplicates())
within_cell_proportion_df['sum_cell_gene'] = (
    within_cell_proportion_df
    .groupby(['cell_barcode','gene'])
    ['viral_bc_UMIs']
    .transform('sum'))
within_cell_proportion_df['proportion_viral_bcs_cell_gene'] = (
    within_cell_proportion_df['viral_bc_UMIs'] /
    within_cell_proportion_df['sum_cell_gene'])

# Rank viral barcodes by proportion within each cell-gene
within_cell_proportion_df['viral_bc_rank'] = (
    within_cell_proportion_df
    .groupby(['cell_barcode','gene'])
    ['proportion_viral_bcs_cell_gene']
    .rank(ascending=False, method='first'))

# Order cells by proportion of max viral barcode
cell_bc_ranks = (within_cell_proportion_df
                 .query('viral_bc_rank == 1')
                 .groupby(['gene','cell_barcode'])
                 ['proportion_viral_bcs_cell_gene']
                 .max()
                 .reset_index())
cell_bc_ranks['cell_rank'] = (cell_bc_ranks
                              .groupby('gene')
                              ['proportion_viral_bcs_cell_gene']
                              .rank(ascending=False, method='first'))
within_cell_proportion_df = pd.merge(
    left=within_cell_proportion_df,
    right=cell_bc_ranks[['gene','cell_barcode','cell_rank']],
    how='left',
    validate='many_to_one'
)

Set limit and annotate each viral barcode:

In [ ]:
within_cell_limit = 0.05
within_cell_proportion_df['above_within_cell_limit'] = (
    within_cell_proportion_df['proportion_viral_bcs_cell_gene'] >
    within_cell_limit)
display(within_cell_proportion_df)

In [ ]:
fig = (p9.ggplot(within_cell_proportion_df,
                 p9.aes(x='proportion_viral_bcs_cell_gene',
                        fill='factor(above_within_cell_limit)')) +
       p9.geom_histogram(position='stack', binwidth=0.05) +
       p9.geom_vline(xintercept=within_cell_limit,
                     linetype='dashed',
                     color=CBPALETTE[2]) +
       p9.facet_grid('gene~') +
       p9.ggtitle('infected cells with multiple valid viral barcodes\n'
                  'ordered by max viral barcode proportion\n'
                  f'{expt}') +
       p9.labs(x='proportion of viral barcode UMIs in cell',
               y='n viral barcodes') +
       p9.theme(figure_size=(3, 4),
                plot_title=p9.element_text(size=9),
                axis_title=p9.element_text(size=9),
                axis_text=p9.element_text(rotation=45),
                legend_title=p9.element_text(size=9),
                legend_title_align='center') +
       p9.scale_fill_manual(CBPALETTE[0:]))
display(fig)

In [ ]:
fig = (p9.ggplot(within_cell_proportion_df,
                 p9.aes(x='cell_rank',
                        y='proportion_viral_bcs_cell_gene',
                        color='factor(above_within_cell_limit)')) +
       p9.geom_point(alpha=0.2) +
       p9.geom_hline(yintercept=within_cell_limit,
                     linetype='dashed',
                     color=CBPALETTE[2]) +
       p9.facet_grid('gene~') +
       p9.ggtitle('infected cells with multiple valid viral barcodes\n'
                  'ordered by max viral barcode proportion\n'
                  f'{expt}') +
       p9.labs(x='infected cell #',
               y='proportion of viral barcode UMIs in cell') +
       p9.theme(figure_size=(3, 4),
                plot_title=p9.element_text(size=9),
                axis_title=p9.element_text(size=9),
                legend_title=p9.element_text(size=9),
                legend_title_align='center') +
       p9.scale_color_manual(CBPALETTE[0:]))
display(fig)

Merge annotations into `viral_bc_frac` dataframe.

In [ ]:
viral_bc_frac = pd.merge(left=viral_bc_frac,
                         right=(within_cell_proportion_df
                                [['cell_barcode',
                                  'gene',
                                  'viral_barcode',
                                  'proportion_viral_bcs_cell_gene',
                                  'above_within_cell_limit']]),
                         on=['cell_barcode','gene','viral_barcode'],
                         how='outer',
                         validate='many_to_one')
viral_bc_frac['above_within_cell_limit'] = (
    viral_bc_frac['above_within_cell_limit']
    .fillna(False)
)
display(viral_bc_frac)

Add within-cell proportion to filter query:

In [ ]:
filter_query_list.append('(above_within_cell_limit == True)')
filter_query = " and ".join(filter_query_list)
print("The filter query is:  "
      f"{filter_query}")

## Assign filter status to each viral barcode

In [ ]:
viral_bc_frac.loc[viral_bc_frac.eval(filter_query), 'valid_viral_bc'] = True
viral_bc_frac['valid_viral_bc'] = viral_bc_frac['valid_viral_bc'].fillna(False)
display(viral_bc_frac)

## Summary visualizations
### Valid viral barcodes in each infected cell

In [ ]:
valid_rank = (viral_bc_frac
              .query('infected == "infected"')
              [['cell_barcode',
                'gene',
                'frac_viral_bc_UMIs',
                'valid_viral_bc']]
              .copy()
              .drop_duplicates())
valid_rank['rank'] = (valid_rank
                      .groupby('gene')
                      ['frac_viral_bc_UMIs']
                      .rank(ascending=False, method='first'))

fig = (p9.ggplot(valid_rank,
                 p9.aes(x='rank',
                        y='frac_viral_bc_UMIs',
                        color='valid_viral_bc')) +
       p9.geom_point(alpha=.3) +
       p9.facet_grid('gene~') +
       p9.scale_y_log10() +
       p9.ggtitle('valid and filterd viral barcodes\n'
                  'in infected cells\n'
                  f'{expt}') +
       p9.labs(x='viral bc frequency rank') +
       p9.theme(figure_size=(3, 4),
                plot_title=p9.element_text(size=9),
                axis_title=p9.element_text(size=9),
                legend_title=p9.element_text(size=9),
                legend_title_align='center') +
       p9.scale_color_manual(CBPALETTE[0:]))
display(fig)

### Number of valid viral barcodes per cell

In [ ]:
n_viral_bc = (viral_bc_frac
             .query('(valid_viral_bc == True)',
                    engine='python')
             .groupby(['cell_barcode',
                       'gene',
                       'valid_viral_bc'])
             ['viral_barcode']
             .nunique()
             .reset_index()
             .rename(columns={'viral_barcode': 'n_viral_bc'}))

In [ ]:
output_fig = (p9.ggplot(n_viral_bc) +
              p9.geom_histogram(p9.aes(x='n_viral_bc'),
                                       binwidth=1,
                                       position='dodge') +
              p9.facet_grid('~gene') +
              p9.ggtitle('Number of valid viral barcodes per cell\n'
                         'in infected cells\n'
                         f'{expt}') +
              p9.xlab('number of valid viral barcodes in cell') +
              p9.ylab('number of cells') +
              p9.theme(figure_size=(5, 2),
                       plot_title=p9.element_text(size=11),
                       axis_title=p9.element_text(size=10),
                       legend_title=p9.element_text(size=10),
                       legend_title_align='center'))
display(output_fig)

In [ ]:
print(f"Saving plots to {plot}")
p9.ggsave(plot=output_fig, filename=plot, verbose=False)

print(f"Saving filtered barcodes to {valid_viral_barcodes_csv}")

# Export valid cell barcode-viral barcodes pairs
(viral_bc_frac
 .query('valid_viral_bc == True')
 .to_csv(valid_viral_barcodes_csv,
         columns=['cell_barcode',
                  'gene',
                  'viral_barcode',
                  'valid_viral_bc'],
         index=False))